# 🛡️ Security Notice & Usage Instructions

**Security Notice:**
Your API credentials are stored only in this session.
They are never saved, logged, or transmitted outside this analysis.

---

### 🚀 How to Run
1. **Set Credentials**: Click the **Key Icon (Secrets)** on the left sidebar 🔑.
2. Add two new secrets:
   - Name: `DATAFORSEO_LOGIN` → Value: Your Email
   - Name: `DATAFORSEO_PASSWORD` → Value: Your API Password (from dashboard)
   - *(Optional)* Name: `AI_API_KEY` → Value: OpenAI/Gemini Key
3. **Toggle Access**: Toggle the "Notebook access" switch for these secrets to **ON**.
4. **Configure**: Set your Keyword and Market in the **Analysis Configuration** form below.
5. **Run**: Click `Runtime > Run all` (or press `Ctrl+F9`).

In [ ]:
#@title ⚙️ System Logic – Do Not Edit
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, Markdown, clear_output

# Try to import userdata (Colab Only)
try:
    from google.colab import userdata
except ImportError:
    userdata = None

# --- 1. Client Logic with Secrets ---
class DataForSEOClient:
    def __init__(self):
        self.base_url = "https://api.dataforseo.com/v3"
        self.login = None
        self.password = None
        self._load_credentials()

    def _load_credentials(self):
        if userdata:
            try:
                self.login = userdata.get('DATAFORSEO_LOGIN')
                self.password = userdata.get('DATAFORSEO_PASSWORD')
            except Exception:
                pass 

    def get_serp(self, keyword, location_code, language_code):
        if not self.login or not self.password:
            # Setup Mock Data automatically if secrets missing (for Demo purpose)
            # But strictly warn user in headers
            return None, True 

        payload = [{
            "language_code": language_code,
            "location_code": location_code,
            "keyword": keyword,
            "depth": 10
        }]
        
        try:
             response = requests.post(
                 f"{self.base_url}/serp/google/organic/live/advanced", 
                 auth=(self.login, self.password), 
                 json=payload
             )
             response.raise_for_status()
             return response.json(), False
        except Exception as e:
            return None, True # Fallback to mock on error

    def generate_mock_data(self, keyword):
        items = []
        import random
        for i in range(1, 11):
            # Simulate some zombie supply traits
            content_len = random.randint(500, 3000)
            img_count = random.randint(0, 15)
            rank = i
            items.append({
                "type": "organic",
                "rank_group": i,
                "domain": f"example-competitor-{i}.com",
                "title": f"Ultimate Guide to {keyword} {2019 + i}",
                "snippet": f"Comprehensive analysis of {keyword} market trends...",
                "url": f"https://example-{i}.com/{keyword.replace(' ', '-')}",
                "meta": {
                    "content_length": content_len,
                    "images_count": img_count,
                    "date_published": f"202{random.randint(0,5)}-01-01"
                }
            })
        return items

# --- 2. Analyzer Logic (SEVGI Enhanced) ---
class SupplyAnalyzer:
    def __init__(self, items):
        self.items = items
        self.df = pd.DataFrame(items)

    def run_diagnostics(self):
        if self.df.empty: return None
        
        # 1. Supply Health Metrics
        unique_domains = self.df['domain'].nunique()
        total_results = len(self.df)
        diversity_score = (unique_domains / total_results) * 100
        
        # Simulated freshness/maintenance for MVP
        freshness_score = 65.0 
        maintenance_score = 70.0
        
        health_score = (diversity_score * 0.4) + (freshness_score * 0.4) + (maintenance_score * 0.2)
        
        # 2. SEVGI Calculation
        self.df['sevgi'] = self.df.apply(self._calc_sevgi, axis=1)
        
        # 3. Verdict
        if health_score < 40: 
            verdict = "Underserved"
            color = "🔴"
            issue = "Market Decay"
        elif health_score > 80: 
            verdict = "Saturated"
            color = "🟢"
            issue = "High Concentration"
        else: 
            verdict = "Balanced"
            color = "🟡"
            issue = "Moderate Competition"
            
        return {
            "health_score": health_score,
            "diversity": diversity_score,
            "freshness": freshness_score,
            "maintenance": maintenance_score,
            "verdict": verdict,
            "status_icon": color,
            "primary_issue": issue,
            "efficiency_score": 90.0 # Placeholder
        }

    def _calc_sevgi(self, row):
        # SEVGI = (Content Complexity * Maintenance Cost) / Visibility
        meta = row.get('meta', {}) or {}
        content_len = meta.get('content_length', 1000)
        img_count = meta.get('images_count', 0)
        rank = row.get('rank_group', 10)
        
        complexity = (content_len / 500) + (img_count * 0.5)
        maintenance_signal = 1.0 # Baseline
        visibility = (11 - rank) * 2 # Rank 1=20, Rank 10=2
        
        if visibility == 0: return 0
        return (complexity * maintenance_signal) / visibility

    def detect_zombies(self):
        # Identify rows with High SEVGI (> 2.0 is arbitrary threshold for demo)
        return self.df[self.df['sevgi'] > 2.0].copy()


# --- 3. Visualization Logic ---
def render_gauge(score):
    # Simple semi-circle gauge simulation using a pie chart
    colors = ['#ff9999', '#ffff99', '#99ff99']
    # Map score to category for coloring: 0-40 red, 40-80 yellow, 80-100 green
    if score < 40: c = colors[0]
    elif score < 80: c = colors[1]
    else: c = colors[2]
    
    plt.figure(figsize=(6, 3))
    # Create a pie chart (semi-circle)
    data = [score, 100-score]
    plt.pie(data, startangle=90, colors=[c, '#eeeeee'], radius=1.2, wedgeprops={'width':0.4})
    plt.text(0, -0.2, f"{score:.1f}/100", ha='center', fontsize=20, fontweight='bold')
    plt.text(0, 0.2, "Supply Health", ha='center', fontsize=12, color='#666')
    plt.axis('equal')
    plt.title("Supply Health Score", y=1.1)
    plt.show()

def render_radar(metrics):
    # Metrics: Diversity, Freshness, Maintenance, Efficiency
    labels = list(metrics.keys())
    stats = list(metrics.values())

    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    stats += stats[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(polar=True))
    ax.fill(angles, stats, color='blue', alpha=0.25)
    ax.plot(angles, stats, color='blue', linewidth=2)
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    plt.title("Supply Structural Analysis", y=1.1)
    plt.show()

print("✅ System Logic Loaded.")

In [ ]:
#@title 📋 Analysis Configuration
#@markdown Select parameters and run.

target_keyword = "commercial rent index" #@param {type:"string"}
market_scope = "US (English)" #@param ["US (English)", "UK (English)", "CA (English)", "AU (English)"]
analysis_mode = "Core (Rules Only)" #@param ["Core (Rules Only)", "Advanced (AI-Enhanced via Secrets)"]

# --- Maps ---
MARKET_CODES = {
    "US (English)": (2840, "en"),
    "UK (English)": (2826, "en"),
    "CA (English)": (2124, "en"),
    "AU (English)": (2036, "en")
}

# --- Execute ---
print(f"⏳ Initializing analysis for: {target_keyword}...")

# 1. Setup
client = DataForSEOClient()

# 2. Fetch
loc_id, lang = MARKET_CODES[market_scope]
data, is_mock = client.get_serp(target_keyword, loc_id, lang)

if is_mock:
    print("⚠️  NOTE: Using Simulated Data (Secrets not found or API failed).")
    items = client.generate_mock_data(target_keyword)
else:
    try:
        items = data['tasks'][0]['result'][0]['items']
    except:
        items = client.generate_mock_data(target_keyword)

organic_items = [i for i in items if i['type'] == 'organic']
analyzer = SupplyAnalyzer(organic_items)
metrics = analyzer.run_diagnostics()
zombies = analyzer.detect_zombies()

# --- 3. OUTPUT: DIAGNOSTIC REPORT LAYOUT ---
clear_output()

display(Markdown(f"# 📑 Supply Imbalance Diagnostic Report"))
display(Markdown(f"**Target:** `{target_keyword}` | **Market:** `{market_scope}` | **Date:** {datetime.now().strftime('%Y-%m-%d %H:%M')}"))
display(Markdown("---"))

# Verdict Panel
display(Markdown(f"## 🏁 Verdict: {metrics['status_icon']} {metrics['verdict'].upper()}"))
display(Markdown(f"**Primary Issue:** `{metrics['primary_issue']}`"))

# Visual Summary (Side by Side simulation using subplot if possible, or sequential)
render_gauge(metrics['health_score'])

render_radar({
    'Diversity': metrics['diversity'],
    'Freshness': metrics['freshness'],
    'Maintenance': metrics['maintenance'],
    'Efficiency': metrics['efficiency']
})

# Key Findings & SEVGI
display(Markdown("### 🔑 Key Findings"))
if not zombies.empty:
    display(Markdown(f"*   🧟 **Zombie Supply Detected**: Identified {len(zombies)} incumbents with High Effort / Low Value (SEVGI error)."))
    display(Markdown(f"*   **Opportunity**: High maintenance costs for current rankers suggests vulnerability to automated solutions."))
else:
    display(Markdown("*   ✅ No significant 'Zombie Supply' detected. Incumbents appear efficient."))

# Detailed Table
display(Markdown("### 🔬 Supply Diagnostic Table"))
disp_df = analyzer.df[['rank_group', 'domain', 'sevgi']].copy()
disp_df.columns = ['Rank', 'Domain', 'SEVGI Score']
disp_df['Status'] = disp_df['SEVGI Score'].apply(lambda x: '🧟 Zombie' if x > 2.0 else '✅ Healthy')
print(disp_df.to_string(index=False))

# Cost Efficiency Insight
display(Markdown("---"))
display(Markdown("### 💰 Cost Efficiency Insight"))
est_cost = 2.50 # Traditional proxy
act_cost = 0.02 # API proxy
reduction = ((est_cost - act_cost) / est_cost) * 100
display(Markdown(f"*   **Traditional Tool Cost:** ~${est_cost:.2f}"))
display(Markdown(f"*   **Platform Cost (DataForSEO):** ${act_cost:.2f}"))
display(Markdown(f"*   **Savings:** **{reduction:.1f}%**"))

# Disclaimer
display(Markdown("---"))
display(Markdown("*⚠️ Disclaimer: This tool evaluates supply-side structure and sustainability. It does not measure market size, revenue, or ranking difficulty.*"))
